In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import pprint

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [9]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count
print(len(cities))


15


In [23]:
#create a df of lats and longs
city_df=pd.DataFrame(cities, columns=['City'])
city_df.rename(columns={0:'New'})

#add new fields to df
def add_fields(x):
    x['Temperature (F)']=""
    x['Humitidy (%)']=""
    x['Cloudiness (%)']=""
    x['Wind Speed (mph)']=""
    x['Lattitude']=""
    x['Longitude']=""

add_fields(city_df)#apply function

city_df.head()

,City,Temperature (F),Humitidy (%),Cloudiness (%),Wind Speed (mph),Lattitude,Longitude
0,guia lopes da laguna,,,,,,
1,hilo,,,,,,
2,gamba,,,,,,
3,rikitea,,,,,,
4,talara,,,,,,


## Perform API Calls

In [11]:


# OpenWeatherMap API Key
api_key = api_keys.api_key

# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key #+f"&q={city_df['City'][0]}"
url

'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=48c662d8e2fbc87db3c1601e94215d84'

In [47]:
# response = requests.get(url)
# r=response.json()
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key
missing=0
# use iterrows to iterate through pandas dataframe
for index, row in city_df.iterrows():
    
    # get restaurant type from df
    city_to_find = row['City']

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {city_to_find}.")
    response = requests.get(base_url+f'&q={city_to_find}').json()

    try:
        #print(response)#(f"Temp is results['main']['temp'] in row['City'].")

        city_df.loc[index,'Temperature (F)'] = response['main']['temp']
        city_df.loc[index,'Lattitude'] = response['coord']['lat']
        city_df.loc[index,'Longitude'] = response['coord']['lon']
        city_df.loc[index,'Humitidy (%)'] = response['main']['humidity']
        city_df.loc[index,'Cloudiness (%)'] = response['clouds']['all']
        city_df.loc[index,'Wind Speed (mph)'] = response['wind']['speed']
        
    except (KeyError, IndexError):
        missing +=1
print(f'missing: {missing}')
print('Start ' +str(len(cities)))
city_df.dtypes

missing: 1
Start 15


City                object
Temperature (F)     object
Humitidy (%)        object
Cloudiness (%)      object
Wind Speed (mph)    object
Lattitude           object
Longitude           object
dtype: object

In [48]:
city_df.dtypes


City                object
Temperature (F)     object
Humitidy (%)        object
Cloudiness (%)      object
Wind Speed (mph)    object
Lattitude           object
Longitude           object
dtype: object

In [49]:
city_df[['Temperature (F)','Humitidy (%)','Cloudiness (%)','Wind Speed (mph)','Lattitude','Longitude']].apply(pd.to_numeric)
city_df.dtypes

City                object
Temperature (F)     object
Humitidy (%)        object
Cloudiness (%)      object
Wind Speed (mph)    object
Lattitude           object
Longitude           object
dtype: object

In [50]:
# xax=city_df['Lattitude']
# yax=city_df['Temperature (F)']
# plt.scatter(xax, yax)

In [ ]:
#ToDo Collect: Temp (F) from each city, Humidity(%), Cloudiness (%),Wind speed (mph) from each city
# Maybe use itterrows???
#include a print log (maybe write to txt file)

#graph info

#save figs

#Save df as csv


#3 trends and analysis

In [46]:
#print(city_df.head())
#city_df.loc[city_df['Cloudiness (%)'] ==44]
#city_df['Longitude'].convert_objects(convert_numeric=True)
#city_df['Longitude'].apply(pd.to_numeric)
#city_df.loc[city_df['Longitude']>-0.6]

#city_df.loc[city_df['Longitude']>90]
#city_df['Longitude'].convert_objects
#city_df['Longitude'] = pd.to_numeric(city_df['Longitude'])

# def convert_to_float(x,y):
#     try:
#         pd.to_numeric(x[y])
#     except:
#         next
# new_df= city_df.apply(convert_to_float)


city_df[['Lattitude','Longitude']].apply(pd.to_numeric)
city_df.dtypes
#how do i create a function to convert all cols to float AND keep str columns???

City                object
Temperature (F)     object
Humitidy (%)        object
Cloudiness (%)      object
Wind Speed (mph)    object
Lattitude           object
Longitude           object
dtype: object

In [44]:
city_df.dtypes


City                object
Temperature (F)     object
Humitidy (%)        object
Cloudiness (%)      object
Wind Speed (mph)    object
Lattitude           object
Longitude           object
dtype: object

In [ ]:
city_df['Longitude'].convert_objects

In [ ]:
city_df.dtypes